# Ingest and track data from pipeline runs

We've now seen how individual datasets can be ingested, let's move on to ingesting datasets generated by a pipeline run. 

```{note}

For the purpose of this guide, we ingest the pipeline output from within this notebook. Typically, this is done from the command line.
```

In [ ]:
import lamindb as ln
import lamindb.schema as lns

ln.nb.header()

## Ingest raw data

In [ ]:
filepath = ln.dev.datasets.file_fastq()

filepath

Create a BFX pipeline:

In [ ]:
pipeline = ln.add(lns.Pipeline(v="1", name="10x scRNA-seq nextseq"))

In [ ]:
pipeline

And a pipeline run:

In [ ]:
run = lns.Run(pipeline=pipeline, name="ingest-fastq")

In [ ]:
run

We see the run points to the pipeline:

In [ ]:
run.pipeline

Let us ingest data from this pipeline run.

In [ ]:
dobject_fq = ln.DObject(filepath, source=run)

In [ ]:
dobject_fq

In [ ]:
dobject_fq.source

In [ ]:
dobject_fq = ln.add(dobject_fq)

We can now select dobject by `run`:

In [ ]:
ln.select(ln.DObject).join(lns.Run, name="ingest-fastq").df()

## Ingest and track pipeline outputs

In [ ]:
output_filepath = ln.dev.datasets.file_bam()

output_filepath

Let's now register another pipeline, which will use cellranger to analyze the scRNA-seq data from the input fastq file.

In [ ]:
pipeline = ln.add(lns.Pipeline(v="7", name="Cell Ranger v7"))
run = lns.Run(pipeline=pipeline, name="cellranger scRNA-seq")

run

```{note}

Linking run input files will allow data lineage tracking.
```

In [ ]:
run.inputs.append(dobject_fq)

In [ ]:
dobject = ln.DObject(output_filepath, source=run)

ln.add(dobject)

## Track data lineage

Now let's track from which files that the `output.bam` file is generated, aka, the input file of the run that produced file `output.bam`

In [ ]:
ss = ln.Session()

run = ss.select(lns.Run).join(ln.DObject, name="output", suffix=".bam").one()

In [ ]:
run.inputs

In [ ]:
ss.close()